In [1]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
%matplotlib qt
import matplotlib.image as mtlimg


# 1. Find corners of Chessboard


In [2]:
def get_chess_image_point_corner_pts(num_row, num_col, glob_path, is_plot=False, is_debug=False):
    # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
    objp = np.zeros((num_col*num_row,3), np.float32)
    objp[:,:2] = np.mgrid[0:num_row, 0:num_col].T.reshape(-1,2)
    # Arrays to store object points and image points from all the images.
    objpoints = [] # 3d points in real world space
    imgpoints = [] # 2d points in image plane.

    # Make a list of calibration images
    images = glob.glob(glob_path)
    print ("found " + str(len(images)) + " image")
    # Step through the list and search for chessboard corners
    for idx, fname in enumerate(images):
        
        if is_debug: print ("Processing " + fname, end = ', ')
        img = cv2.imread(fname)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Find the chessboard corners
        ret, corners = cv2.findChessboardCorners(gray, (num_row, num_col), None)

        # If found, add object points, image points
        if ret == True:
            if is_debug: print ("found corner:" + str(num_row) + "x" + str(num_col), end= " ,")
            print (".", end=" ")    
            objpoints.append(objp)
            imgpoints.append(corners)

            if is_plot:
                # Draw and display the corners
                cv2.drawChessboardCorners(img, (num_row ,num_col), corners, ret)
                cv2.imshow('img', img)
                cv2.waitKey(500)

    cv2.destroyAllWindows() 
    return objpoints, imgpoints
    
#def calibate_camera(glob_path, ):

In [3]:
glob_path = 'camera_cal/calibration*.jpg'

In [4]:
objpoints, imgpoints = get_chess_image_point_corner_pts(9, 6, glob_path, is_plot=False, is_debug=False)
print ("done")

found 24 image
. . . . . . . . . . . . . . . . . . done


# 2. Calibrate and UnDistort image

In [5]:
def calibrate_camera(objpoints, imgpoints, img):
    img_size = (img.shape[1], img.shape[0])

    # Do camera calibration given object points and image points
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size, None,None)
    cal_dict = {}
    cal_dict['mtx'] = mtx
    cal_dict['dist'] = dist
    return cal_dict

def un_distort_image(img, cal_dict):
    dst = cv2.undistort(img, cal_dict['mtx'], cal_dict['dist'], None, cal_dict['mtx'])
    return dst

def get_un_distort_from_distorted_image(test_image_path, objpoints, imgpoints):
    img = cv2.imread(test_image_path)
    calibration_dict = calibrate_camera(objpoints, imgpoints, img)
    img_un_dist = un_distort_image(img, calibration_dict)
    return img, img_un_dist, calibration_dict
    
def plot_two_images(img1, img2, left_title="Left Image", right_title="Right image"):
    f, axes = plt.subplots(1, 2, figsize=(20,10))
    axes[0].imshow(img1, cmap='gray')
    axes[0].set_title(left_title, fontsize=20)
    axes[1].imshow(img2, cmap='gray')
    axes[1].set_title(right_title, fontsize=20)    
    
def plot_distorted_image_un_distorted_image(image_path, objpoints, imgpoints, is_plot=False):
    img, img_un_dist, calibration_dict = get_un_distort_from_distorted_image(image_path, objpoints, imgpoints)
    if is_plot:
        plot_two_images(img, img_un_dist, left_title="Original image", right_title="Undistorted image")
    return img, img_un_dist, calibration_dict


In [6]:
_ = plot_distorted_image_un_distorted_image("camera_cal/calibration1.jpg", objpoints, imgpoints, is_plot=True)

In [31]:
_  = plot_distorted_image_un_distorted_image("camera_cal/calibration2.jpg", objpoints, imgpoints, is_plot=True)

done


In [19]:
_ = plot_distorted_image_un_distorted_image("camera_cal/calibration3.jpg", objpoints, imgpoints, is_plot=True)

In [7]:
cal_image = cv2.imread("camera_cal/calibration1.jpg")
cal_dict = calibrate_camera(objpoints, imgpoints, cal_image)

# 3. Step to perform Perspective Transform

In [8]:
def get_perspective_transform(src, dest, undist_image):
    M = cv2.getPerspectiveTransform(src, dest)
    img_size = (undist_image.shape[1], undist_image.shape[0])
    warped = cv2.warpPerspective(undist_image, M, img_size, flags=cv2.INTER_LINEAR)
    return M, warped

In [9]:
perspective_image_path = "test_images/straight_lines2.jpg"
pesp_img = cv2.imread(perspective_image_path)
pesp_img_rgb = cv2.cvtColor(pesp_img, cv2.COLOR_BGR2RGB)

In [14]:
%matplotlib qt

persp_test_image = mtlimg.imread(perspective_image_path)
print (persp_test_image.shape)
plt.imshow(persp_test_image)
plt.plot(207, 720, "+")
plt.plot(1100, 720, ".")
plt.plot(667, 440, ".")
plt.plot(611, 440, ".")


(720, 1280, 3)


In [13]:
src_pnts = np.float32([
  [207, 720],
  [1100, 720],
  [667, 440],    
  [611, 440]

])
dst_pnts = np.float32([
  [207, 720],
  [1100, 720],
  [1100, 110],    
  [207,  110]
])

# corners = np.float32([[190,720],[589,457],[698,457],[1145,720]])
# new_top_left=np.array([corners[0,0],0])
# new_top_right=np.array([corners[3,0],0])
# offset=[150,0]
    
img_size = (persp_test_image.shape[1], persp_test_image.shape[0])
# src_pnts = np.float32([corners[0],corners[1],corners[2],corners[3]])
# dst_pnts = np.float32([corners[0]+offset,new_top_left+offset,new_top_right-offset ,corners[3]-offset])    
    
img_un_dist= un_distort_image(persp_test_image, cal_dict)
M, warped = get_perspective_transform(src_pnts, dst_pnts, persp_test_image)
img_rgb = cv2.cvtColor(pesp_img, cv2.COLOR_BGR2RGB)
warped_rgb = cv2.cvtColor(warped, cv2.COLOR_BGR2RGB)

plot_two_images(img_rgb, warped_rgb)


In [15]:
cv2.destroyAllWindows()

# 3. Apply color transfor and image gradients 

In [73]:
# Define a function to return the magnitude of the gradient
# for a given sobel kernel size and threshold values
def magnitute_thresh(img, sobel_kernel=3, mag_thresh=(0, 255)):
    # Apply the following steps to img
    # 1) Convert to grayscale
    # 2) Take the gradient in x and y separately
    # 3) Calculate the magnitude 
    # 4) Scale to 8-bit (0 - 255) and convert to type = np.uint8
    # 5) Create a binary mask where mag thresholds are met
    # 6) Return this mask as your binary_output image
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Take both Sobel x and y gradients
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Calculate the gradient magnitude
    gradmag = np.sqrt(sobelx**2 + sobely**2)
    # Rescale to 8 bit
    scale_factor = np.max(gradmag)/255 
    gradmag = (gradmag/scale_factor).astype(np.uint8) 
    # Create a binary image of ones where threshold is met, zeros otherwise
    binary_output = np.zeros_like(gradmag)
    binary_output[(gradmag >= mag_thresh[0]) & (gradmag <= mag_thresh[1])] = 1

    # Return the binary image
    return binary_output

# Define a function to threshold an image on directon for a given range and Sobel kernel
def direction_threshold(img, sobel_kernel=3, dir_thresh=(0, np.pi/2)):
    # Grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Calculate the x and y gradients
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Take the absolute value of the gradient direction, 
    # apply a threshold, and create a binary image result
    absgraddir = np.arctan2(np.absolute(sobely), np.absolute(sobelx))
    binary_output =  np.zeros_like(absgraddir)
    binary_output[(absgraddir >= dir_thresh[0]) & (absgraddir <= dir_thresh[1])] = 1

    # Return the binary image
    return binary_output

In [67]:
mag_thres_test = magnitute_thresh(test_img, mag_thresh=(70, 230))
plot_2_images(test_img, mag_thres_test)

In [83]:
dir_thres_test = direction_threshold(test_img, dir_thresh=(0, 2* np.pi))
plot_2_images(test_img, dir_thres_test)